Two high-level analyses:
- Predict performance (accuracy) using a regression
- Classify users into 'bad' vs 'good'--still need to define this but perhaps the threshold is any user < 70% and any user >= 70%

For github issues and brainstorming features and analyses, use github:
- https://github.com/ProjectSidewalk/sidewalk-quality-analysis/issues

In [11]:
%%time
import pandas as pd
users = pd.read_csv('ml-users.csv') 
#display(users)

Wall time: 3 ms
Wall time: 3 ms


In [3]:
# the interactions table should really have the following header (but doesn't currently)
# audit_task_interaction_id, audit_task_id, action, gsv_panorama_id, lat, lng, heading, pitch, zoom, note, 
# timestamp, temporary_label_id, mission_id 

# I had to turn off exceptions because of parsing problems, including:
# https://github.com/ProjectSidewalk/sidewalk-quality-analysis/issues/3
# col_names = ['audit_task_interaction_id', 'audit_task_id', 'action', 'gsv_panorama_id', 'lat', 'lng', 
#            'heading', 'pitch', 'zoom', 'note', 'timestamp', 'temporary_label_id', 'mission_id']
# interactions = pd.read_csv('ml-interactions5000.csv', error_bad_lines=False,
#                           header=None, names = col_names, index_col=False) 
#interactions = pd.read_csv('ml-interactions5000.csv')

In [17]:
%%time
interactions = pd.read_csv('ml-interactions.tar.gz', compression='gzip') # load the full gzip tarball
#interactions = pd.read_csv('ml-interactions15000.csv')
display(interactions)

,ml-interactions.csv,user_id,audit_task_interaction_id,audit_task_id,action,gsv_panorama_id,lat,lng,heading,pitch,zoom,note,temporary_label_id,mission_id
0,2019-04-25 18:56:51.007-07,bf16418a-4c99-4fd6-99c6-7e8b40fbe17b,10829973.0,3458.0,LowLevelEvent_mousemove,yswXpbti65Rk8pcuuB5IAQ,47.629299,-122.301750,178.535721,-6.598214,2.0,"cursorX:600,cursorY:421",NaN,2740.0
1,2019-04-25 18:56:51.102-07,bf16418a-4c99-4fd6-99c6-7e8b40fbe17b,10830000.0,3458.0,POV_Changed,yswXpbti65Rk8pcuuB5IAQ,47.629299,-122.301750,183.178574,-6.598214,2.0,NaN,NaN,2740.0
2,2019-04-25 18:56:56.289-07,2d9009b3-55d5-4aa8-a17a-a7c80afc4d51,10830056.0,3463.0,POV_Changed,Br6DzMb4z7VHPJoqJv-w5Q,47.680912,-122.319862,174.540176,-18.908482,1.0,NaN,NaN,2747.0
3,2019-04-25 18:56:51.985-07,bf16418a-4c99-4fd6-99c6-7e8b40fbe17b,10830121.0,3458.0,LowLevelEvent_mouseout,yswXpbti65Rk8pcuuB5IAQ,47.629299,-122.301750,183.178574,-6.598214,2.0,"cursorX:641,cursorY:163",NaN,2740.0
4,2019-04-25 18:56:56.523-07,2d9009b3-55d5-4aa8-a17a-a7c80afc4d51,10830133.0,3463.0,LowLevelEvent_mousemove,Br6DzMb4z7VHPJoqJv-w5Q,47.680912,-122.319862,151.477676,-20.220982,1.0,"cursorX:709,cursorY:426",NaN,2747.0
5,2019-04-25 18:56:52.019-07,bf16418a-4c99-4fd6-99c6-7e8b40fbe17b,10830146.0,3458.0,LowLevelEvent_mousemove,yswXpbti65Rk8pcuuB5IAQ,47.629299,-122.301750,183.178574,-6.598214,2.0,"cursorX:604,cursorY:142",NaN,2740.0
6,2019-04-25 18:56:56.623-07,2d9009b3-55d5-4aa8-a17a-a7c80afc4d51,10830158.0,3463.0,LowLevelEvent_mousemove,Br6DzMb4z7VHPJoqJv-w5Q,47.680912,-122.319862,151.477676,-20.220982,1.0,"cursorX:533,cursorY:425",NaN,2747.0
7,2019-04-25 18:56:56.767-07,2d9009b3-55d5-4aa8-a17a-a7c80afc4d51,10830186.0,3463.0,LowLevelEvent_mousemove,Br6DzMb4z7VHPJoqJv-w5Q,47.680912,-122.319862,148.290176,-21.158482,1.0,"cursorX:549,cursorY:419",NaN,2747.0
8,2019-04-25 18:56:56.808-07,2d9009b3-55d5-4aa8-a17a-a7c80afc4d51,10830198.0,3463.0,POV_Changed,Br6DzMb4z7VHPJoqJv-w5Q,47.680912,-122.319862,138.352676,-22.470982,1.0,NaN,NaN,2747.0
9,2019-04-25 18:56:52.185-07,bf16418a-4c99-4fd6-99c6-7e8b40fbe17b,10830210.0,3458.0,LowLevelEvent_mousemove,yswXpbti65Rk8pcuuB5IAQ,47.629299,-122.301750,183.178574,-6.598214,2.0,"cursorX:517,cursorY:134",NaN,2740.0


Wall time: 1min 11s


In [21]:
#interactions.head(100)
interactions[['audit_task_interaction_id', 'audit_task_id']].head()

,audit_task_interaction_id,audit_task_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [13]:
interactions.describe()

,audit_task_interaction_id,audit_task_id,lat,lng,heading,pitch,zoom,temporary_label_id,mission_id
count,1.850278e+07,1.850278e+07,1.850100e+07,1.850100e+07,1.850100e+07,1.850100e+07,1.850100e+07,3.972245e+06,1.850278e+07
mean,2.373016e+07,8.721192e+03,4.756261e+01,-1.219587e+02,1.796261e+02,-1.941253e+01,1.338253e+00,2.198867e+02,4.872183e+03
std,1.006763e+07,3.961152e+03,7.830798e-01,4.068231e+00,1.061091e+02,1.089005e+01,5.890745e-01,2.256418e+02,1.804001e+03
min,1.000000e+00,1.000000e+00,3.894045e+01,-1.224168e+02,0.000000e+00,-3.500000e+01,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.640451e+07,5.778000e+03,4.758485e+01,-1.223460e+02,8.975000e+01,-2.862500e+01,1.000000e+00,4.700000e+01,3.769000e+03
50%,2.308122e+07,8.578000e+03,4.764685e+01,-1.223230e+02,1.789464e+02,-1.997322e+01,1.000000e+00,1.440000e+02,4.776000e+03
75%,3.320987e+07,1.237200e+04,4.767582e+01,-1.222995e+02,2.705960e+02,-1.143750e+01,2.000000e+00,3.160000e+02,6.579000e+03
max,4.073552e+07,1.527500e+04,4.773420e+01,-7.700277e+01,3.600000e+02,0.000000e+00,3.000000e+00,1.184000e+03,8.045000e+03


In [20]:
# Let's analyse keyboard usage as a signal for performance with the hypotheses that power users will use
# keyboard shortcuts and power users tend to be better performers

# the following is currently throwing an exception because at least one row has NaN or null for
# the 'action' cell. need to explore this further and figure out what's going on and then, perhaps, eliminate this row
#interactions[interactions['action'].str.contains('Keyboard')]

df = interactions
result = df[df['action'] == ''].index
null_df = pd.isna(df['action'])
display(null_df)

0           False
1           False
2           False
3           False
4           False
5           False
6           False
7           False
8           False
9           False
10          False
11          False
12          False
13          False
14          False
15          False
16          False
17          False
18          False
19          False
20          False
21          False
22          False
23          False
24          False
25          False
26          False
27          False
28          False
29          False
            ...  
18502750    False
18502751    False
18502752    False
18502753    False
18502754    False
18502755    False
18502756    False
18502757    False
18502758    False
18502759    False
18502760    False
18502761    False
18502762    False
18502763    False
18502764    False
18502765    False
18502766    False
18502767    False
18502768    False
18502769    False
18502770    False
18502771    False
18502772    False
18502773    False
18502774  